In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
import tensorflow as tf
from seldon.tensorflow_wrapper import TensorFlowWrapper
from sklearn.pipeline import Pipeline
import seldon.pipeline.util as sutl

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cmap
import matplotlib.gridspec as gridspec
%matplotlib inline

nh = 6
nv = 4

gs = gridspec.GridSpec(nv, nh, top=1., bottom=0., right=1., left=0., hspace=0.,
        wspace=0.)

for g,im in zip(gs,mnist.train.images[:nv*nh]):
    ax = plt.subplot(g)
    ax.imshow(im.reshape((28,28)),cmap=cmap.Greys)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('auto')


In [ ]:
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,name=name)

def bias_variable(shape,name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial,name=name)

def conv2d(x, W, name):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME',name=name)

def max_pool_2x2(x,name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

## Creating Model

In [ ]:
x = tf.placeholder(tf.float32, [None,784], name='x')

W_conv1 = weight_variable([5, 5, 1, 32], 'W_conv1')
b_conv1 = bias_variable([32], 'b_conv1')

x_image = tf.reshape(x, [-1,28,28,1], name='x_image')

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1,'conv1') + b_conv1, name='h_conv1')
h_pool1 = max_pool_2x2(h_conv1, name='h_pool1')

W_conv2 = weight_variable([5, 5, 32, 64], 'W_conv2')
b_conv2 = bias_variable([64], 'b_conv2')

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 'conv2') + b_conv2, name='h_conv2')
h_pool2 = max_pool_2x2(h_conv2, name='h_pool2')

W_fc1 = weight_variable([7 * 7 * 64, 1024], 'W_fc1')
b_fc1 = bias_variable([1024], 'b_fc1')

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name='h_pool2_flat')
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name='h_fc1')

keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name='h_fc1_drop')

W_fc2 = weight_variable([1024, 10], 'W_fc2')
b_fc2 = bias_variable([10], 'b_fc2')

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y_conv')

y_ = tf.placeholder(tf.float32, [None, 10], name='y_')


cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]), name='cross_entropy')
tf.scalar_summary('cross entropy', cross_entropy)

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1), name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
tf.scalar_summary('accuracy', accuracy)

merged_summaries = tf.merge_all_summaries()
init = tf.initialize_all_variables()

sess = tf.Session()

train_writer = tf.train.SummaryWriter('summaries/train',sess.graph)
test_writer = tf.train.SummaryWriter('summaries/test')


In [ ]:
sess.run(init)

## Training Model

In [ ]:
for i in range(5000):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if i%20 == 0:
            summary,test_accuracy = sess.run([merged_summaries,accuracy],feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob:1.0})
            test_writer.add_summary(summary, i)
            print("step %d, test accuracy %.3f"%(i, test_accuracy))
        summary,_ = sess.run([merged_summaries,train_step], feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
        train_writer.add_summary(summary,i)

print("test accuracy %g"%sess.run(accuracy,feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

## Creating and Saving Seldon Pipeline

In [ ]:
tfw = TensorFlowWrapper(sess,tf_input=x,tf_output=y_conv,tf_constants=[(keep_prob,1.0)],target="y",target_readable="class",excluded=['class'])

p = Pipeline([('deep_classifier',tfw)])

saver = sutl.PipelineWrapper()

saver.save_pipeline(p,'tensorflow_deep_mnist_model')

## Starting TensorBoard

In [ ]:
# To execute in a terminal window (blocking process)
%%bash
tensorboard --logdir=summaries --port=25002 --reload_interval=2